In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import osmnx as ox
import folium 
import branca
from scipy import spatial
from scipy.spatial import distance 
from geopy.distance import geodesic

In [2]:
import math
# This definition was taken from here: 
# https://www.timvink.nl/closest-coordinates/
def cartesian(latitude, longitude, elevation = 0):
    # Convert to radians
    latitude = latitude * (math.pi / 180)
    longitude = longitude * (math.pi / 180)

    R = 6371 # 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(latitude) * math.cos(longitude)
    Y = R * math.cos(latitude) * math.sin(longitude)
    Z = R * math.sin(latitude)
    return (X, Y, Z)



def deg2rad(degree):
    '''
    function to convert degree to radian
    '''
    rad = degree * 2*np.pi / 360
    return(rad)

def rad2deg(rad):
    '''
    function to convert radian to degree
    '''
    degree = rad/2/np.pi * 360
    return(degree)


def distToKM(x):
    '''
    function to convert cartesian distance to real distance in km
    '''
    R = 6371 # earth radius
    gamma = 2*np.arcsin(deg2rad(x/(2*R))) # compute the angle of the isosceles triangle
    dist = 2*R*math.sin(gamma/2) # compute the side of the triangle
    return(dist)

def kmToDIST(x):
    '''
    function to convert real distance in km to cartesian distance 
    '''
    R = 6371 # earth radius
    gamma = 2*np.arcsin(x/2./R) 
    
    dist = 2*R*rad2deg(math.sin(gamma / 2.))
    return(dist)

#### Acquire the POI data

In [3]:
# Put here the coordinates of the centre of the area of analysis
# For Sydney: 
c_point = (-33.882882592547034, 151.2063073174587)
# For Zurich:
#c_point = (47.372133, 8.516359)
# Change the distance to fit the boundaries of the desired area of analysis
# The distance should be the same as the one used to acquire street network data in the file 
# 'Building the spatial database --Step 2 -- Street network data acquisition from OSM'
poi_df = ox.pois.pois_from_point((c_point[0],c_point[1]),dist=1000,
                                 tags={'amenity':True, 'landuse':['commercial','retail']})
poi_df

,osmid,geometry,highway,ref,start_date,element_type,guide_text,guide_type,guidepost,information,...,display,min_height,roof:colour,roof:shape,tower:construction,tower:type,visibility,source:addr,source:boundary,level:ref
12501605,12501605,POINT (151.19782 -33.87640),traffic_signals,1836,1980-05-23,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12501610,12501610,POINT (151.20129 -33.88137),NaN,NaN,NaN,node,Taylor Square,destination,bicycle,guidepost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12501974,12501974,POINT (151.20122 -33.88231),traffic_signals,NaN,NaN,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13757642,13757642,POINT (151.20391 -33.88078),NaN,NaN,NaN,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13767762,13767762,POINT (151.19777 -33.87796),NaN,NaN,NaN,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786589750,786589750,"POLYGON ((151.20103 -33.88148, 151.20096 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
786604871,786604871,"POLYGON ((151.19776 -33.87709, 151.19814 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
809336368,809336368,"POLYGON ((151.20791 -33.87964, 151.20778 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
809336375,809336375,"POLYGON ((151.20620 -33.88001, 151.20587 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Calculate the latitude and longitude for each POI 

In [4]:
poi_df['Latitude']=0
poi_df['Longitude']=0

for index in poi_df.index:
    i =poi_df.loc[index, 'geometry']
    this_type = str(i).split(' ')[0]
    lat=0
    lon=0
    if this_type=='POINT':
        lat = float(str(i).split(' ')[2].replace(')',''))
        lon = float(str(i).split(' ')[1].replace('(',''))

    if (this_type=='POLYGON')|(this_type=='MULTIPOLYGON'):
        all_pts = str(i).split(' ')[1:]
        lat_list = []
        lon_list = []
        for pt in range(0, len(all_pts[:2]), 2):
            this_pt = all_pts[pt:pt+2]
            lat = float(this_pt[1].replace(')','').replace(',',''))
            lon = float(this_pt[0].replace('(','').replace(',',''))
    poi_df.loc[index, 'Latitude']=lat
    poi_df.loc[index, 'Longitude']=lon
poi_df


,osmid,geometry,highway,ref,start_date,element_type,guide_text,guide_type,guidepost,information,...,roof:colour,roof:shape,tower:construction,tower:type,visibility,source:addr,source:boundary,level:ref,Latitude,Longitude
12501605,12501605,POINT (151.19782 -33.87640),traffic_signals,1836,1980-05-23,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.876402,151.197821
12501610,12501610,POINT (151.20129 -33.88137),NaN,NaN,NaN,node,Taylor Square,destination,bicycle,guidepost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.881373,151.201287
12501974,12501974,POINT (151.20122 -33.88231),traffic_signals,NaN,NaN,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.882310,151.201215
13757642,13757642,POINT (151.20391 -33.88078),NaN,NaN,NaN,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.880777,151.203912
13767762,13767762,POINT (151.19777 -33.87796),NaN,NaN,NaN,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.877964,151.197770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786589750,786589750,"POLYGON ((151.20103 -33.88148, 151.20096 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.881478,151.201029
786604871,786604871,"POLYGON ((151.19776 -33.87709, 151.19814 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.877092,151.197763
809336368,809336368,"POLYGON ((151.20791 -33.87964, 151.20778 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.879644,151.207909
809336375,809336375,"POLYGON ((151.20620 -33.88001, 151.20587 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.880007,151.206203


#### Visualise the acquired POI data

In [5]:
poi_map = folium.Map(location = [c_point[0],c_point[1]], zoom_start = 20, tiles='CartoDB dark_matter' ) 
poi_map.save("POI map.html" )

node_latitudes = poi_df['Latitude'].values.tolist()
node_longitudes = poi_df['Longitude'].values.tolist()



for item in list(zip(node_latitudes,node_longitudes)):
    folium.CircleMarker([item[0],item[1]], color='red', fill=True, radius=0.5, opacity=1,
               popup = 'node').add_to(poi_map)

    
poi_map.save("POI map.html" )
from IPython.display import IFrame

IFrame(src='./POI map.html', width=1500, height=1500)

In [6]:
places = []
for index, row in poi_df.iterrows():
    coordinates = [row['Latitude'], row['Longitude']]
    cartesian_coord = cartesian(*coordinates)
    places.append(cartesian_coord)

#### Construct a kd-tree describing the spatial proximity of the poi data

In [7]:
tree = spatial.KDTree(places)

#### Calculate POI density

In [8]:
kd_closest = []

poi_df['Density']=0


for i in range(0, len(poi_df.index)):
    item = poi_df.index[i]

    this_pt = (poi_df.loc[item, 'Latitude'],poi_df.loc[item, 'Longitude'])
    closest = tree.query([cartesian(this_pt[0],this_pt[1])], p = 2, k=100)
    
    lats = []
    lons = []
    for c in closest[1][0]:
       
        lats.append(poi_df.loc[poi_df.index[c], 'Latitude'])
        lons.append(poi_df.loc[poi_df.index[c], 'Longitude'])
    dist_to_other_points = np.array(distance.cdist(np.array([this_pt]), list(zip(lats,lons)), lambda u, v: geodesic(u, v).meters))
    poi_df.loc[item, 'Density']=len(np.where(dist_to_other_points[0]<=100)[0])
poi_df
    


,osmid,geometry,highway,ref,start_date,element_type,guide_text,guide_type,guidepost,information,...,roof:shape,tower:construction,tower:type,visibility,source:addr,source:boundary,level:ref,Latitude,Longitude,Density
12501605,12501605,POINT (151.19782 -33.87640),traffic_signals,1836,1980-05-23,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.876402,151.197821,11
12501610,12501610,POINT (151.20129 -33.88137),NaN,NaN,NaN,node,Taylor Square,destination,bicycle,guidepost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.881373,151.201287,17
12501974,12501974,POINT (151.20122 -33.88231),traffic_signals,NaN,NaN,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.882310,151.201215,26
13757642,13757642,POINT (151.20391 -33.88078),NaN,NaN,NaN,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.880777,151.203912,9
13767762,13767762,POINT (151.19777 -33.87796),NaN,NaN,NaN,node,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.877964,151.197770,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786589750,786589750,"POLYGON ((151.20103 -33.88148, 151.20096 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.881478,151.201029,15
786604871,786604871,"POLYGON ((151.19776 -33.87709, 151.19814 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.877092,151.197763,19
809336368,809336368,"POLYGON ((151.20791 -33.87964, 151.20778 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.879644,151.207909,8
809336375,809336375,"POLYGON ((151.20620 -33.88001, 151.20587 -33.8...",NaN,NaN,NaN,way,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33.880007,151.206203,21


#### Visualise POI density

In [9]:
poi_density_map = folium.Map(location = [c_point[0],c_point[1]], zoom_start = 20, tiles='CartoDB dark_matter' ) 
poi_density_map.save("POI_density_map.html" )

for item in poi_df.index:
    this_pt = (poi_df.loc[item, 'Latitude'],poi_df.loc[item, 'Longitude'])
    this_density = poi_df.loc[item, 'Density']
    
    this_color='red'

    if this_density>=50:
        this_color='darkpurple'
    elif (this_density>40) & (this_density<=50):
        this_color='purple'
    elif (this_density>30) & (this_density<=40):
        this_color='darkred'
    elif (this_density>20) & (this_density<=30):
        this_color='red'
    elif (this_density>10) & (this_density<=20):
        this_color = 'orange'
    elif (this_density>5) & (this_density<=10):
        this_color = 'beige'
    else:
        this_color = 'green'
            
            
    folium.CircleMarker([this_pt[0],this_pt[1]], color=this_color, fill=True, radius=1, opacity=0.5,
                   popup = str(this_density)).add_to(poi_density_map)
                                                   

poi_density_map.save("POI_density_map.html" )
from IPython.display import IFrame

IFrame(src='./POI_density_map.html', width=1500, height=1500)


#### Save this file to a folder that has all the files related to the spatial database 

In [10]:
poi_df.to_csv(r'C:\Users\demdr\Desktop\Testing the thesis functions\Project data\Spatial database\POI_df.csv')